## SnowPit Data - XML data to python dataframe
#### Alexis Emerson

In [1]:
import xmltodict
import pandas as pd

In [2]:
file_path = "../raw_data/sp/all-MT.xml"
with open(file_path, "rb") as file:
    pit_data = xmltodict.parse(file)

In [4]:
# pit_data

In [16]:
df = pd.DataFrame.from_dict(pit_data['Pit_Data']['Pit_Observation'])
df.columns = [col.replace('@', '') for col in df.columns]
df.set_index('nid', inplace=True)

In [17]:
df.head()

,activities,iLayerNumber,iDepth,bld,stability,aviPit,crownObs,aviLoc,incline,winDir,...,Location,Layer,Shear_Test_Result,Density_Profile,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits
nid,,,,,,,,,,,,,,,,,,,,,
45338,,4,65,7.91,Good,false,false,,17,E,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'CT', '@sdepth': '65.0', '@score': ...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45310,Ski tracks on slope; We skied slope;,2,64,7.91,Good,false,false,,,SW,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '20.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45286,Recent avalanche activity on similar slopes; R...,6,50,7.91,Poor,false,false,,25,SW,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '64.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45347,,6,22,7.91,,false,false,,10,S,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '98.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45346,,6,22,7.91,,false,false,,10,S,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '98.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN


In [18]:
# layers = df.Layer
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)
# layers

In [19]:
# ml = {'@grainSizeUnits1': 'mm', '@grainSizeUnits2': 'mm', '@grainSize': '', '@grainSize1': '', '@grainType': '', '@grainType1': '', '@hardness1': 'F', '@hardness2': '', '@layerNumber': '1', '@waterContent': 'D', '@startDepth': '110', '@endDepth': '100', '@multipleHardness': 'false', '@multipleGrainType': 'false', '@multipleGrainSize': 'false'}
# pd.DataFrame.from_dict(ml)

In [25]:
dflayers = pd.DataFrame()

for i,layers in df.Layer.items():
    if isinstance(layers, dict):
        layers = [layers]
    ph = pd.DataFrame(layers)
    ph['nid'] = i
#     dflayers = pd.concat([layers, ph], axis='rows')

ValueError: DataFrame constructor not properly called!